In [ ]:
import pandas as pd
import numpy as np

# Import library for visualisasi
import matplotlib.pyplot as plt
import seaborn as sns

# Import library for machine learning
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error


In [ ]:
# Load the dataset from the specified file location and display the first few rows
dataset_location = "File Location"
df = pd.read_csv(dataset_location)
df.head()

In [ ]:
#Find Outlier
#Plot sales distribution using boxplot
plt.figure(figsize=(19,8))
plt.boxplot(df.Sales)
plt.title('Sales Distribution: Boxplot Analysis')
plt.show()

In [ ]:
#Plot sales distribution
plt.figure(figsize=(19,8))
plt.hist(df['Sales'], bins=20,color='skyblue',edgecolor='black')
plt.title('Sales Distribution')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.tight_layout
plt.show()

In [ ]:
#Plot sales distribution by categories
grouped_data = df.groupby('Category')['Sales'].sum().reset_index()
labels = grouped_data['Category']
sizes = grouped_data['Sales']
plt.figure(figsize=(19,8))
plt.pie(sizes, labels=labels,autopct='%1.1f%%')
plt.title('Sales Distribution by Categories')
plt.show()

In [ ]:
#Plot average sales per category
data_histogram= df.groupby('Category')['Sales'].mean().reset_index().sort_values(by='Sales', ascending=False)
data_histogram = data_histogram.head(3)
plt.figure(figsize=(19,8))
plt.bar(data_histogram['Category'], data_histogram['Sales'])
plt.xlabel('Category')
plt.ylabel('Avarage Sales')
plt.title('Average Sales per Category')
plt.show()

In [ ]:
df.info()

In [ ]:
#Filter the DataFrame to include only the 'Furniture' category and sort it by 'Sales' in descending order
furniture_data=df[df['Category']=='Furniture'].sort_values(by='Sales', ascending=False)

#Display summary statistics for the filtered Furniture sales data
furniture_data.describe()

In [ ]:
#Filter the DataFrame to include only the 'Technology' category and sort it by 'Sales' in descending order
technology_data=df[df['Category']=='Technology'].sort_values(by='Sales', ascending=False)

#Display summary statistics for the filtered technology sales data
technology_data.describe()

In [ ]:
#Filter the DataFrame to include only the 'Office Supplies' category and sort it by 'Sales' in descending order
office_supplies_data = df[df['Category']=='Office Supplies'].sort_values(by='Sales', ascending=False)

#Display summary statistics for the filtered Office Supplies sales data
office_supplies_data.describe()

In [ ]:
#Remove outliers in the 'Sales' column by filtering for value between 1 and 470
df=df[(df['Sales']>=1)&(df['Sales']<=470)]

In [ ]:
#Display basic information about dataset, including data types and non-null counts
df.info()

In [ ]:
#Check for missing (null) values in the DataFrame and display the count of null values for each column
df.isnull().sum()

In [ ]:
#Check for Duplicate rows in the DataFrame and display the count of duplicate
df.duplicated().sum()

In [ ]:
# Replace missing values in the 'Postal Code' column with 0
df['Postal Code'].fillna(0, inplace=True)

In [ ]:
# Display summary statistics for the Data Frame
df.describe()

In [ ]:
#Plot sales distribution using boxplot
sns.boxplot(y=df['Sales'])

In [ ]:
# Convert the 'Order Date' and 'Ship Date' columns to datetime data type
# Then, sort the DataFrame based on 'Order Date' in descending order
df['Order Date'] = pd.to_datetime(df['Order Date'])
df['Ship Date'] = pd.to_datetime(df['Ship Date'])
df = df.sort_values(by='Order Date', ascending=False)

FEATURE ENGINEERING

In [ ]:
#Make a new column for day, month, year, week and dayname
df['Day'] = (df['Order Date']).dt.day
df['Month'] = (df['Order Date']).dt.month
df['Year'] = (df['Order Date']).dt.year
df['Week'] = (df['Order Date']).dt.isocalendar().week
df['Dayname'] = df['Order Date'].dt.day_name()

In [ ]:
#Mapping months to season
season_mapping = {
    1: 'Winter',
    2: 'Winter',
    3: 'Spring',
    4: 'Spring',
    5: 'Spring',
    6: 'Summer',
    7: 'Summer',
    8: 'Summer',
    9: 'Fall',
    10: 'Fall',
    11: 'Fall',
    12: 'Winter'
}

df["Season"]=df['Month'].map(season_mapping)

EXPLORATORY DATA ANALYSIS

In [ ]:
#Plot yearly sales trends
data_group_sales_by_month = df.groupby(['Year','Month'])['Sales'].sum().reset_index().sort_values(by=['Year','Month'])
data_group_sales_by_month['YearMonth'] = data_group_sales_by_month['Year'].astype(str) + '-' + data_group_sales_by_month['Month'].astype(str)
data_group_sales_by_month['YearMonth'] = pd.to_datetime(data_group_sales_by_month['YearMonth'] + '-1', format='%Y-%m-%d')
plt.figure(figsize=(19,8))
sns.lineplot(x='YearMonth', y='Sales', data=data_group_sales_by_month)
plt.title('Year-to-Year Sales Growth Trends')
plt.show()

In [ ]:
#plot sales distribution by product categories
grouped_data = df.groupby('Category')['Sales'].sum().reset_index()
labels = grouped_data['Category']
sizes = grouped_data['Sales']
plt.figure(figsize=(19,8))
plt.pie(sizes, labels=labels,autopct='%1.1f%%')
plt.title('Sales Distribution by Product Categories')
plt.show()

In [ ]:
#plot sales distribution by customer segments
grouped_data = df.groupby('Segment')['Sales'].sum().reset_index()
labels = grouped_data['Segment']
sizes = grouped_data['Sales']
plt.figure(figsize=(19,8))
plt.pie(sizes, labels=labels,autopct='%1.1f%%')
plt.title('Sales Distribution by Customer Segments')
plt.show()

In [ ]:
#Plot top 10 revenue-generating cities
top_10_city_based_on_revenue_df=df.groupby('City')['Sales'].sum().reset_index().sort_values(by='Sales', ascending=False)

plt.figure(figsize=(26,10))
plt.barh(top_10_city_based_on_revenue_df['City'].head(10)[::-1], top_10_city_based_on_revenue_df['Sales'].head(10)[::-1], color='tomato')
plt.xlabel('Total Sales')
# plt.ylabel('City')
plt.title('Top 10 Revenue-Generating Cities')

for sales,city in zip(top_10_city_based_on_revenue_df['Sales'].head(10)[::-1], top_10_city_based_on_revenue_df['City'].head(10)[::-1]):
    plt.text(sales,city,f'${sales: .2f}',horizontalalignment='left',color='black',fontsize=12)

plt.show()

In [ ]:
#Plot shipping methods comparison
ship_mode_ranking_based_on_revenue_df = df.groupby('Ship Mode')['Sales'].sum().reset_index().sort_values(by='Sales', ascending=False)
plt.figure(figsize=(12,6))
plt.bar(ship_mode_ranking_based_on_revenue_df['Ship Mode'], ship_mode_ranking_based_on_revenue_df['Sales'], color='tomato')
plt.xlabel('Ship Mode')
plt.ylabel('Total Sales')
plt.title('Shipping Methods Comparison')

for k,v in enumerate(ship_mode_ranking_based_on_revenue_df['Sales']):
    plt.text(k,v,'$'+str(v), fontsize=12 ,color='black',horizontalalignment='center',verticalalignment='bottom')


plt.show()

In [ ]:
#Plot top 10 revenue-generating product
top_10_product_df= df.groupby('Product Name')['Sales'].sum().reset_index().sort_values(by='Sales', ascending=False)
plt.figure(figsize=(19,8))
plt.barh(top_10_product_df['Product Name'].head(10)[::-1], top_10_product_df['Sales'].head(10)[::-1],color='skyblue')
plt.xlabel('Total Sales')
# plt.ylabel('Total Sales')
plt.title('Top 10 Revenue-Generating Product')

for sales, product in zip(top_10_product_df['Sales'].head(10)[::-1],top_10_product_df['Product Name'].head(10)[::-1]):
     label =f'${sales:.2f}'
     plt.text(sales,product, label, horizontalalignment='center',verticalalignment='center',color='black',fontsize=12)

plt.show()

In [ ]:
#Plot average daily sales
dayname_mean_sales_df = df.groupby('Dayname')['Sales'].mean().reset_index()
day_order = pd.DataFrame({
    'Dayname': ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'],
    'Day_Order': [0, 1, 2, 3, 4, 5, 6]
})

dayname_mean_sales_df =dayname_mean_sales_df.merge(day_order, on='Dayname', how='left')

dayname_mean_sales_df = dayname_mean_sales_df.sort_values(by='Day_Order', ascending=True)

plt.figure(figsize=(19,8))
plt.bar(dayname_mean_sales_df['Dayname'], dayname_mean_sales_df['Sales'],color='tomato')
plt.ylabel('Average Sales')
plt.title('Average Daily Sales')

for day,sales in zip(dayname_mean_sales_df['Dayname'], dayname_mean_sales_df['Sales']):
    label =f'${sales:.2f}'
    plt.text(day, sales, label, horizontalalignment='center',verticalalignment='bottom',color='black',fontsize=12)

plt.show()

In [ ]:
#Plot seasonal sales analysis
season_order = ['Winter', 'Spring', 'Summer', 'Fall']
seasonal_sales_analysis = df.groupby('Season')['Sales'].sum().reindex(season_order).reset_index()
plt.figure(figsize=(19,8))
plt.bar(seasonal_sales_analysis['Season'],seasonal_sales_analysis['Sales'], color='tomato')
plt.xlabel('Season')
plt.ylabel('Total Sales')
plt.title('Seasonal Sales Analysis')

for season, sales in zip(seasonal_sales_analysis['Season'],seasonal_sales_analysis['Sales']):
    plt.text(season, sales, f'{sales:.2f}', horizontalalignment='center',verticalalignment='bottom',color='black', fontsize=12)

plt.show()

In [ ]:
#Plot Top 10 Sub-Category by revenue
data_top_5_sub_category_by_sales = df.groupby('Sub-Category')['Sales'].sum().reset_index().sort_values(by='Sales', ascending=False)
plt.figure(figsize=(19,8))
plt.bar(data_top_5_sub_category_by_sales['Sub-Category'].head(10), data_top_5_sub_category_by_sales['Sales'].head(10),color='tomato')
plt.xlabel('Sub-Category')
plt.ylabel('Total Sales')
plt.title('Top 10 Revenue-Generating Sub-Categories')

for subcategory,sales in zip(data_top_5_sub_category_by_sales['Sub-Category'].head(10), data_top_5_sub_category_by_sales['Sales'].head(10)):
    plt.text(subcategory, sales, f'{sales:.2f}',horizontalalignment='center', verticalalignment='bottom', color='black', fontsize=12)

plt.show()


In [ ]:
#Plot Most Valuable Customer
loyal_customer_df = df.groupby('Customer Name')['Sales'].sum().reset_index().sort_values(by='Sales',ascending=False)
plt.figure(figsize=(19,8))
plt.barh(loyal_customer_df['Customer Name'].head(10)[::-1], loyal_customer_df['Sales'].head(10)[::-1], color = 'lightblue')
plt.xlabel('Total Spend')
# plt.ylabel('Customer Name')
plt.title('Most Valuable Customer')

for sales, customer in zip(loyal_customer_df['Sales'].head(10)[::-1],loyal_customer_df['Customer Name'].head(10)[::-1]):
    plt.text(sales, customer, f'${sales:.2f}', horizontalalignment= 'center', verticalalignment='center',fontsize=12)

plt.show()

In [ ]:
#Plot Sales Distribution
plt.figure(figsize=(19,8))
plt.subplot(1,2,1)
plt.hist(df['Sales'], bins=20,color='skyblue',edgecolor='black')
plt.xlabel('Sales')
plt.ylabel('Frequency')
plt.title('Sales Distribution')
plt.show()

RANDOM FOREST

In [ ]:
df_superstore = df[['Day','Week', 'Month','Year','Category','Sales','Region', 'Sub-Category']]
numerical_feature=['Day','Week', 'Month','Year','Sales']
categorical_feature = ['Category','Region', 'Sub-Category']

#encode categorical feature using one-hot encoding
encoded_categorical_feature = pd.get_dummies(df_superstore[categorical_feature], drop_first=True)

#concatenate the encoded feature with the numerical feature
encoded_df_superstore= pd.concat([encoded_categorical_feature, df_superstore[numerical_feature]], axis=1)

#Splitting data into feature(X) and target(y)
X = encoded_df_superstore.drop('Sales', axis=1)
y =encoded_df_superstore['Sales']

#Splitting data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
#initialize the model
model = RandomForestRegressor()

In [ ]:
#fit the model to your data
model.fit(X_train,y_train)

In [ ]:
#make prediction
y_prediction = model.predict(X_test)

In [ ]:
#evaluate the model using mse
mse_RF = mean_squared_error(y_test,y_prediction, squared=False)
print('mean squared error:', mse_RF)

In [ ]:
#evaluate the model using mae
mae_RF = mean_absolute_error(y_test, y_prediction)
print('mean absolute error:', mae_RF)

In [ ]:
#evaluate the model using rmse
rmse_RF = np.sqrt(mse_RF)
print('Root Mean Squared Error :', rmse_RF)

RANDOM FOREST HYPERPARAMETER TUNING

In [ ]:
#Define the Parameter grid for GridSearchCV
n_estimators = [100,200,300]
max_feature = [10,15,20,26]
min_samples_leaf = [100,250,500,1000] 
max_depth = [10,15,25]
param_grid = {'n_estimators': n_estimators,'max_features': max_feature, 'min_samples_leaf': min_samples_leaf,'max_depth': max_depth}

In [ ]:
#Perform Hyperparameter tuning using GridSearchCV
grid_search = GridSearchCV(estimator=model,param_grid= param_grid, cv=5, n_jobs=-1)
grid_search.fit(X_train, y_train)


In [ ]:
#Print the best grid parameters
print("best grid parameters:", grid_search.best_params_)

In [ ]:
grid_search.score(X_train,y_train)

In [ ]:
#Fit a Random Forest model with the best hyperparameters
rf_tuning = RandomForestRegressor(
    n_estimators=grid_search.best_params_['n_estimators'], 
    max_features=grid_search.best_params_['max_features'], 
    min_samples_leaf=grid_search.best_params_['min_samples_leaf'], 
    max_depth=grid_search.best_params_['max_depth']
)

rf_tuning.fit(X_train,y_train)

In [ ]:
#Make predictions
y_prediction_tuning = rf_tuning.predict(X_test)

In [ ]:
#Evaluate the model using mean squared error(mse)
mse_RFTuning = mean_squared_error(y_test,y_prediction_tuning, squared=False)
print('mean squared error:', mse_RFTuning)

In [ ]:
#Evaluate the model using mean absolute error(mae)
mae_RFTuning = mean_absolute_error(y_test, y_prediction_tuning)
print('mean absolute error:', mae_RFTuning)

In [ ]:
#Evaluate the model using root mean squared error(rmse)
rmse_RFTuning = np.sqrt(mse_RFTuning)
print('Root Mean Squared Errot:', rmse_RFTuning)

In [ ]:
#Create a comparison DataFrame for model evaluation
comparison_model_df = pd.DataFrame({
    'Model':['Random Forest', 'Random Forest With Hyperparameter Tuning'],
    'MSE': [mse_RF,mse_RFTuning],
    'MAE': [mae_RF,mae_RFTuning],
    'RMSE': [rmse_RF,rmse_RFTuning]
})

#Display the comparison DataFrame
comparison_model_df

TESTING THE MODEL WITH THE FIRST 20 DATA POINTS IN X_TEST

In [ ]:
#Testing the model with the first 20 data points in X_test
X_test_subset = X_test.head(20)
prediction_final = rf_tuning.predict(X_test_subset)
result_df = X_test_subset[['Day', 'Month', 'Year']].copy()
result_df['Date'] = pd.to_datetime(X_test_subset[['Day', 'Month', 'Year']])
result_df['Sales'] = y_test.head(20)
result_df['Predicted Sales']= prediction_final
result_df = result_df.sort_values(by='Date', ascending=False)

In [ ]:
result_df.head()

In [ ]:
#Plot actual vs predicted sales
plt.figure(figsize=(19,8))
plt.plot(result_df['Date'], result_df['Sales'], label='Actual Sales', marker='o')
plt.plot(result_df['Date'], result_df['Predicted Sales'], label='Predicted Sales', marker='o')
plt.xlabel("Date")
plt.ylabel('Sales')
plt.title('Actual Sales VS Predicted Sales' )
plt.legend()
plt.show()